In [1]:
import os
import glob
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [3]:
df = pd.read_excel("DATA.xlsx")
df.head()

,TERMS,CATEGORY
0,Partners Capital,CAPITAL ACCOUNT
1,Unsecured Loans,LOAN PAYABLE
2,Current Liabilities Provisions,OTHER LIABILITIES
3,NaN,DROP
4,Fixed Assets,FIXED ASSETS


In [4]:
df.CATEGORY.value_counts()

FIXED ASSETS         64
OTHER LIABILITIES    54
LOAN PAYABLE         48
OTHER ASSETS         46
CASH ACCOUNT         45
CAPITAL ACCOUNT      35
LIQUID CASH          34
DEBTORS              30
STOCK                30
DEPOSITS             30
CREDITORS            29
LOAN RETURN          27
GOLD & JEWELLERY     27
DROP                  2
Name: CATEGORY, dtype: int64

In [6]:
array = ["FIXED ASSETS", "OTHER LIABILITIES", "LOAN PAYABLE", "OTHER ASSETS","CASH ACCOUNT", "CAPITAL ACCOUNT", "LIQUID CASH", "DEBTORS","STOCK", "DEPOSITS","CREDITORS","LOAN RETURN","GOLD & JEWELLERY"]
df2 = df.loc[df['CATEGORY'].isin(array)]

In [7]:
df2 = df2[['TERMS', 'CATEGORY']]
df2.head(2)

,TERMS,CATEGORY
0,Partners Capital,CAPITAL ACCOUNT
1,Unsecured Loans,LOAN PAYABLE


In [8]:
df2["CATEGORY"].isnull().sum()

0

In [9]:
df2['LABEL'] = 0
["STOCK", "DEPOSITS","CREDITORS","LOAN RETURN","GOLD & JEWELLERY"]
df2.loc[df2['CATEGORY'] == 'FIXED ASSETS', 'LABEL'] = 0
df2.loc[df2['CATEGORY'] == 'OTHER LIABILITIES', 'LABEL'] = 1
df2.loc[df2['CATEGORY'] == 'LOAN PAYABLE', 'LABEL'] = 2
df2.loc[df2['CATEGORY'] == 'OTHER ASSETS', 'LABEL'] = 3
df2.loc[df2['CATEGORY'] == 'CASH ACCOUNT', 'LABEL'] = 4
df2.loc[df2['CATEGORY'] == 'CAPITAL ACCOUNT', 'LABEL'] = 5
df2.loc[df2['CATEGORY'] == 'LIQUID CASH', 'LABEL'] = 6
df2.loc[df2['CATEGORY'] == 'DEBTORS', 'LABEL'] = 7
df2.loc[df2['CATEGORY'] == 'STOCK', 'LABEL'] = 8
df2.loc[df2['CATEGORY'] == 'DEPOSITS', 'LABEL'] = 9
df2.loc[df2['CATEGORY'] == 'CREDITORS', 'LABEL'] = 10
df2.loc[df2['CATEGORY'] == 'LOAN RETURN', 'LABEL'] = 11
df2.loc[df2['CATEGORY'] == 'GOLD & JEWELLERY', 'LABEL'] = 12

# print(df2['LABEL'][:10])
labels = to_categorical(df2['LABEL'], num_classes=13)
print(labels[:10])
if 'Mcat' in df2.keys():
    df2.drop(['Mcat'], axis=1)

['STOCK', 'DEPOSITS', 'CREDITORS', 'LOAN RETURN', 'GOLD & JEWELLERY']

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [10]:
df2

,TERMS,CATEGORY,LABEL
0,Partners Capital,CAPITAL ACCOUNT,5
1,Unsecured Loans,LOAN PAYABLE,2
2,Current Liabilities Provisions,OTHER LIABILITIES,1
4,Fixed Assets,FIXED ASSETS,0
5,Loans Advances,LOAN RETURN,11
6,Cash at Bank,CASH ACCOUNT,4
7,Cash in hand per cash book certified by,LIQUID CASH,6
9,Secured Loan,LOAN PAYABLE,2
10,Closing Stock,STOCK,8
11,Rent Deposit,DEPOSITS,9
